# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 18

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
df = pd.read_csv('../data/data_week' + str(gameweek) + '.csv', index_col=0, encoding='latin-1')
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,games played,adjusted points,adjusted points per game,form 5
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,24,100.0,100.0,0,69140,0,0,-3,3,0.8,0,2,1.2,1.2,0,Shkodran,0.7,2,0,7.5,False,19.8,90,NaN,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,4.0,0,0,Mustafi,0.3,False,NaN,a,1,3,54.0,4,7947,42,32342,135,0.1,0.8,Mustafi,0,1.000000,4.475512,4.475512,NaN
2,0,0,26,75.0,75.0,0,98745,0,0,-1,1,22.7,0,2,0.5,0.5,0,HÃÂ©ctor,0.2,6,0,6.9,False,30.8,262,Hamstring injury - 75% chance of playing,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,BellerÃÂ­n,0.3,False,NaN,d,1,3,15.0,1,34674,24,34578,341,0.0,0.2,BellerÃÂ­n,2,3.333333,4.375909,1.312773,1.000168
3,2,1,196,0.0,0.0,1,111457,0,0,-3,3,149.6,0,2,0.0,0.0,0,Sead,1.3,17,0,37.8,False,174.4,928,Ankle injury - Expected back 11 Jan,2019-12-15T19:30:19.136195Z,52,0,0,0,111457.jpg,1.9,0,0,Kolasinac,0.6,False,NaN,i,1,3,55.0,27,46958,11,112204,1076,0.2,5.2,Kolasinac,2,14.210526,38.818566,2.731677,2.486991
4,2,3,150,100.0,100.0,2,154043,0,0,-4,4,143.5,1,2,2.0,2.0,6,Ainsley,1.5,14,0,35.4,False,182.8,760,NaN,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.9,1,0,Maitland-Niles,2.6,False,NaN,a,1,3,33.0,26,548323,1040,569302,593,0.3,5.7,Maitland-Niles,0,8.965517,27.478614,3.064922,3.154755
5,0,2,224,100.0,0.0,2,39476,0,0,-1,1,30.7,0,2,0.8,0.0,0,Sokratis,0.3,23,1,44.3,False,340.2,1350,NaN,2019-12-15T19:30:19.149412Z,49,0,0,0,39476.jpg,2.2,0,0,Papastathopoulos,1.4,False,NaN,a,1,3,71.0,33,107545,179,148456,862,0.1,6.7,Sokratis,5,15.000000,35.841591,2.389439,1.569891


In [4]:
# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [5]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [7]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [8]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [9]:
metric = 'valuePoints metric'
#metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
411,PatrÃÂ­cio,Wolves,52,17.894737,68,78.186538,3.8,4.369248,4.284080,1.916042
471,Henderson,Sheffield Utd,48,16.888889,76,68.617960,4.5,4.062905,5.457301,1.854454
494,Ramsdale,Bournemouth,46,18.000000,72,70.362649,4.0,3.909036,4.597362,1.822597
47,Ryan,Brighton,48,17.948718,70,70.379577,3.9,3.921148,4.145499,1.789751
131,Guaita,Crystal Palace,51,16.136364,71,63.919737,4.4,3.961223,6.186102,1.754059
93,Pope,Burnley,47,18.157895,69,68.643690,3.8,3.780377,3.658800,1.743758
235,de Gea,Manchester Utd,55,18.000000,54,73.082516,3.0,4.060140,3.548081,1.731249
168,Schmeichel,Leicester City,54,17.857143,75,70.983515,4.2,3.975077,3.313907,1.710601
341,Gazzaniga,Tottenham,46,12.142857,34,44.511004,2.8,3.665612,2.994421,1.709100


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
297,Lundstram,Sheffield Utd,52,17.924528,95,93.225552,5.3,5.201004,5.278512,2.280791
182,Alexander-Arnold,Liverpool,72,16.818182,74,89.310307,4.4,5.310343,4.014168,1.979048
208,Otamendi,Manchester City,50,12.105263,46,51.575400,3.8,4.260577,5.010840,1.905388
181,Robertson,Liverpool,70,16.976744,73,84.640916,4.3,4.985698,3.185014,1.884417
185,Matip,Liverpool,52,6.956522,32,29.557788,4.6,4.248932,NaN,1.863280
405,Boly,Wolves,48,8.095238,34,32.499031,4.2,4.014586,NaN,1.832399
401,Doherty,Wolves,61,16.842105,64,75.904372,3.8,4.506822,4.411400,1.824759
164,SÃÂ¶yÃÂ¼ncÃÂ¼,Leicester City,51,17.804878,73,71.167815,4.1,3.997096,3.449786,1.769944
103,Alonso,Chelsea,60,7.966102,47,34.396178,5.9,4.317818,6.851449,1.762742


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
215,De Bruyne,Manchester City,104,16.901408,120,110.333132,7.1,6.528044,6.474742,2.024260
191,Salah,Liverpool,122,14.054054,104,93.542342,7.4,6.655897,9.896777,1.905578
192,ManÃÂ©,Liverpool,122,15.972222,115,101.014566,7.2,6.324390,6.922421,1.810668
196,Keita,Liverpool,58,3.018868,16,13.076462,5.3,4.331578,11.362441,1.798590
431,Pulisic,Chelsea,73,14.042553,66,67.754825,4.7,4.824965,4.323182,1.785800
171,Maddison,Leicester City,78,16.981132,90,84.622798,5.3,4.983343,4.260984,1.784323
287,Cantwell,Norwich City,48,18.205128,71,70.460533,3.9,3.870367,3.897995,1.766573
214,Sterling,Manchester City,118,17.142857,96,101.698595,5.6,5.932418,5.572534,1.726994
29,Grealish,Aston Villa,63,16.086957,74,69.512417,4.6,4.321042,3.790467,1.721544


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
167,Iheanacho,Leicester City,57,3.013699,22,17.605479,7.3,5.841818,NaN,2.446868
166,Vardy,Leicester City,101,17.974684,142,114.863699,7.9,6.390304,7.668943,2.010763
278,Pukki,Norwich City,66,17.924528,95,90.858219,5.3,5.068932,5.375786,1.973079
233,Rashford,Manchester Utd,91,18.103448,105,105.863699,5.8,5.847709,5.312540,1.938497
313,Ings,Southampton,65,18.000000,99,86.752740,5.5,4.819597,6.445262,1.890401
460,Abraham,Chelsea,79,17.118644,101,90.158219,5.9,5.266668,2.897281,1.873796
409,JimÃÂ©nez,Wolves,75,18.039216,92,90.663699,5.1,5.025922,4.854199,1.835207
502,Maupay,Brighton,58,18.000000,72,77.000000,4.0,4.277778,4.160000,1.776251
68,King,Bournemouth,61,13.846154,54,55.858219,3.9,4.034205,2.100706,1.633402


Below we compile a list sorting players based on a given metric (irrespective of position).

In [11]:
metric = 'adjusted points per game'
#metric = 'form 5'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
191,Salah,Liverpool,122,14.054054,104,93.542342,7.4,6.655897,9.896777,1.905578
215,De Bruyne,Manchester City,104,16.901408,120,110.333132,7.1,6.528044,6.474742,2.024260
166,Vardy,Leicester City,101,17.974684,142,114.863699,7.9,6.390304,7.668943,2.010763
192,ManÃÂ©,Liverpool,122,15.972222,115,101.014566,7.2,6.324390,6.922421,1.810668
214,Sterling,Manchester City,118,17.142857,96,101.698595,5.6,5.932418,5.572534,1.726994
233,Rashford,Manchester Utd,91,18.103448,105,105.863699,5.8,5.847709,5.312540,1.938497
167,Iheanacho,Leicester City,57,3.013699,22,17.605479,7.3,5.841818,NaN,2.446868
182,Alexander-Arnold,Liverpool,72,16.818182,74,89.310307,4.4,5.310343,4.014168,1.979048
460,Abraham,Chelsea,79,17.118644,101,90.158219,5.9,5.266668,2.897281,1.873796
